In [1]:
import random
import numpy as np
import pandas as pd
import math
import openpyxl
import time
import sys
np.set_printoptions(threshold=sys.maxsize)
import xlwings
filename1 = "C:\\Users\\admin\\PYTHON\\Thesis_Meta\\GA data.xlsx"
wb1=xlwings.Book(filename1)
sheet1=wb1.sheets[0]

#paramters
period=int(sheet1.range("D3").value)
supermarket=int(sheet1.range("D4").value)
item = int(sheet1.range("D5").value)
pc=sheet1.range("D6:F6").value
hc=sheet1.range("D12:F12").value
oc=int(sheet1.range("D15").value)
WU=int(sheet1.range("D19").value)
I_0=sheet1.range("D21:L21").value
m = 100  #number of chromosomes - population size
n = (3, 3, 12)
shape = (3,3,12)
# Parameter of GA
pselect = 0.2
pmutation = 0.3 
maxI_ga = 100
# Starting time
start_time = time.time()

#Defuzzification 
a=sheet1.range("B27:DE27").value
b=sheet1.range("B28:DE28").value
c=sheet1.range("B29:DE29").value
a = np.array(a)
b = np.array(b)
c = np.array(c)
calculated_values=((a + 2 * b + c)/4)
Demand=np.ceil(calculated_values.reshape((3,3,12)))

def initialize(m, n, p):
    v = np.zeros((m, n, p), dtype=int)
    for i in range(m):
        for j in range(n):
            for k in range(p):
                v[i][j][k] = np.random.randint(0, 2, dtype=int)
    return v
v_list = [initialize(*shape) for _ in range(m)]
print(Demand)

[[[ 8.  7.  4.  6.  6.  8.  5.  6.  8.  8.  7.  4.]
  [ 4.  4.  5.  6.  3.  6.  5.  6.  6.  4.  3.  4.]
  [ 6.  8.  5.  3.  6.  7.  5.  6.  4.  5.  7.  8.]]

 [[ 7.  2.  6.  6.  7.  6.  4.  6.  6.  4.  6.  6.]
  [ 6.  6.  4.  7.  4.  7.  3.  4.  9.  3.  4.  8.]
  [ 5.  6.  7.  4. 10.  6.  8.  6.  6.  2.  3.  5.]]

 [[ 7.  5.  5.  7.  6.  8.  6.  3.  6.  6.  5.  5.]
  [ 7.  7.  3.  9.  4.  6.  3.  5.  6.  4.  6.  6.]
  [ 6. 10.  7.  3.  7.  8.  7.  7.  8.  6.  8.  8.]]]


In [2]:
def check_chromosome(v_list, Demand, WU, initial_inventory):
    modified_v_list = []
    cumulative_Demand_list = []
    for v in v_list:
        v_copy = np.copy(v)
        cumulative_Demand = np.zeros_like(Demand)
        I = np.zeros_like(Demand)  # Inventory levels
        # Initialize inventory levels
        for (i, j), inv in initial_inventory.items():
            I[i, j, 0] = inv

        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                cumulative = 0
                for k in range(v_copy.shape[2]):
                    # Check if initial inventory can cover cumulative demand up to period k
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v_copy[i, j, k] = 0
                        cumulative_Demand[i, j, k] = 0
                        cumulative_Demand[i, j, k+1] = Demand[i, j, k+1]
                        if k+1 < v_copy.shape[2]:
                            v_copy[i, j, k+1] = 1
                    elif I[i, j, 0] < Demand[i, j, 0]:
                        v_copy[i, j, 0] = 1
                        cumulative_Demand[i, j, 0] = Demand[i, j, 0]
                            
                    else:
                        # Adjust based on WU and WL
                        if v_copy[i, j, k] == 1:
                            temp1 = k
                            temp2 = k
                            cumulative_Demand[i, j, temp1] = Demand[i, j, temp1]
                            
                            # Look ahead to find consecutive zeros in v_copy
                            if (k+1) < v_copy.shape[2] and v_copy[i, j, k+1] == 0:
                                while (temp2 + 1) < v_copy.shape[2] and v_copy[i, j, temp2 + 1] == 0:
                                    temp2 += 1
                                cumulative_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])

                        if cumulative_Demand[i, j, k] > WU:
                            v_copy[i, j, k] = 1
                            cumulative_Demand[i, j, k] = Demand[i, j, k]
                       
        modified_v_list.append(v_copy)
        cumulative_Demand_list.append(cumulative_Demand[:, :, :v_copy.shape[2]])  # Trim to match v_copy shape
    return modified_v_list
initial_inventory = {
    (0, 0): 12, (0, 1): 10, (0, 2): 6,
    (1, 0): 18, (1, 1): 10, (1, 2): 0,
    (2, 0): 2,  (2, 1): 5,  (2, 2): 12
}

modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
for i, modified_v in enumerate(modified_v_list, start=1):
    print(f"Modified v[{i}]:\n{modified_v}\n")

Modified v[1]:
[[[0 1 1 0 0 0 0 0 1 1 1 0]
  [0 0 1 0 1 1 0 1 1 1 0 1]
  [0 1 0 1 1 0 0 0 0 0 1 1]]

 [[0 0 0 1 1 0 0 1 0 1 0 0]
  [0 1 1 0 0 1 0 1 1 1 1 0]
  [1 0 0 0 1 0 1 1 0 1 0 0]]

 [[1 0 1 0 0 1 1 0 0 1 1 1]
  [1 1 1 1 1 1 0 0 1 0 1 1]
  [0 1 1 0 1 1 0 1 0 0 0 1]]]

Modified v[2]:
[[[0 1 0 1 0 0 1 1 1 1 1 1]
  [0 0 1 0 0 1 1 1 1 0 1 1]
  [0 1 1 0 0 1 0 1 1 1 1 1]]

 [[0 0 0 1 0 1 0 1 1 0 0 0]
  [0 1 1 1 1 1 0 0 1 0 0 1]
  [1 0 1 0 0 1 0 0 0 1 0 0]]

 [[1 0 0 0 0 0 0 1 0 0 0 1]
  [1 1 1 1 1 1 0 1 1 1 0 1]
  [0 1 1 1 1 0 1 0 1 0 0 1]]]

Modified v[3]:
[[[0 1 0 0 0 1 0 1 0 0 1 0]
  [0 0 1 0 1 1 0 0 0 1 0 1]
  [0 1 1 1 1 0 0 0 0 0 0 1]]

 [[0 0 0 1 0 0 0 1 1 0 0 0]
  [0 1 1 0 0 1 1 1 0 0 0 0]
  [1 1 0 1 0 1 1 0 0 1 0 1]]

 [[1 0 1 1 0 0 0 1 0 0 0 1]
  [1 0 1 0 0 1 1 0 1 0 1 1]
  [0 1 0 1 0 0 1 0 1 0 0 1]]]

Modified v[4]:
[[[0 1 0 1 1 0 0 0 1 1 1 1]
  [0 0 1 0 1 0 0 1 0 0 0 0]
  [0 1 0 0 0 0 0 0 1 1 1 0]]

 [[0 0 0 1 0 0 1 1 0 0 1 1]
  [0 1 1 1 1 1 1 0 0 1 0 1]
  [1 0 0 0 0 0 0 0 0 

In [3]:
def cumuD(v_list, Demand, WU, initial_inventory):
    modified_v_list = []
    cumulative_Demand_list = []
    for v in v_list:
        v_copy = np.copy(v)
        cumulative_Demand = np.zeros_like(Demand)
        I = np.zeros_like(Demand)  # Inventory levels
        # Initialize inventory levels
        for (i, j), inv in initial_inventory.items():
            I[i, j, 0] = inv

        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                cumulative = 0
                for k in range(v_copy.shape[2]):
                    # Check if initial inventory can cover cumulative demand up to period k
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v_copy[i, j, k] = 0
                        cumulative_Demand[i, j, k] = 0
                        cumulative_Demand[i, j, k+1] = Demand[i, j, k+1]
                        if k+1 < v_copy.shape[2]:
                            v_copy[i, j, k+1] = 1
                    elif I[i, j, 0] < Demand[i, j, 0]:
                        v_copy[i, j, 0] = 1
                        cumulative_Demand[i, j, 0] = Demand[i, j, 0]
                            
                    else:
                        # Adjust based on WU and WL
                        if v_copy[i, j, k] == 1:
                            temp1 = k
                            temp2 = k
                            cumulative_Demand[i, j, temp1] = Demand[i, j, temp1]
                            
                            # Look ahead to find consecutive zeros in v_copy
                            if (k+1) < v_copy.shape[2] and v_copy[i, j, k+1] == 0:
                                while (temp2 + 1) < v_copy.shape[2] and v_copy[i, j, temp2 + 1] == 0:
                                    temp2 += 1
                                cumulative_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])

                        if cumulative_Demand[i, j, k] > WU:
                            v_copy[i, j, k] = 1
                            cumulative_Demand[i, j, k] = Demand[i, j, k]
                        
        modified_v_list.append(v_copy)
        cumulative_Demand_list.append(cumulative_Demand[:, :, :v_copy.shape[2]])  # Trim to match v_copy shape
    
    return  cumulative_Demand_list
cumulative_Demand_list = cumuD(v_list, Demand, WU, initial_inventory)
for i, cumulative_Demand in enumerate(cumulative_Demand_list, start=1):
    print(f"Cumulative Demand[{i}]:\n{cumulative_Demand}\n")

Cumulative Demand[1]:
[[[ 0.  7. 35.  0.  0.  0.  0.  0.  8.  8. 11.  0.]
  [ 0.  0. 11.  0.  3. 11.  0.  6.  6.  7.  0.  4.]
  [ 0. 13.  0.  3. 33.  0.  0.  0.  0.  0.  7.  8.]]

 [[ 0.  0.  0.  6. 17.  0.  0. 12.  0. 16.  0.  0.]
  [ 0.  6. 15.  0.  0. 10.  0.  4.  9.  3. 12.  0.]
  [ 5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0. 10. 10.  0.  7. 15.  0. 29.  0.  0.  0.  8.]]]

Cumulative Demand[2]:
[[[ 0. 11.  0. 20.  0.  0.  5.  6.  8.  8.  7.  4.]
  [ 0.  0. 14.  0.  0.  6.  5.  6. 10.  0.  3.  4.]
  [ 0.  8. 14.  0.  0. 12.  0.  6.  4.  5.  7.  8.]]

 [[ 0.  0.  0. 13.  0. 10.  0.  6. 22.  0.  0.  0.]
  [ 0.  6.  4.  7.  4. 14.  0.  0. 16.  0.  0.  8.]
  [ 5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0. 10.  7.  3. 15.  0. 14.  0. 22.  0.  0.  8.]]]

Cumula

In [4]:
for v in modified_v_list:
    for i in range(modified_v_list.shape[0]):
        for j in range(modified_v_list.shape[1]): 
            for k in range(modified_v_list.shape[2]):
                if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                    if k==0:
                        I[i, j, 0] = I[i, j, 0] -Demand[i, j, 0] 
                    else: 
                        I[i, j, k] = I[i, j, k-1] - Demand[i, j, k]
                    if k+1 < modified_v_list.shape[2]:
                        v_copy[i, j, k+1] = 1
                if v_modified_v_list[i, j, k] == 1:
                    temp1 = k
                    temp2 = k

AttributeError: 'list' object has no attribute 'shape'